In [1]:
import json
from urllib.parse import quote
from urllib.parse import quote_plus
import requests
import geopandas as gpd
from shapely.geometry import shape
import numpy as np
import pandas as pd
import numpy as np
import sqlalchemy

In [ ]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
structuresdb = config['db']['structuresdb']

In [ ]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
                                 ';Database=' + structuresdb +
                                 ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
con = engine.connect()

In [23]:
sql = 'select system, shape from parksgis.dpr.structure_evw where bin != 0 and bin % 1000000 > 1 and bin is not null'

In [ ]:
structures = read_geosql(sql,
                         con,
                         geom_raw='shape',
                         geom_col='geometry')

In [17]:
def get_lpc_indivdual(bin, geom_col):
    failed = []
    lpc = []
    parent_url = 'https://data.cityofnewyork.us/api/views/ts56-fkf5'
    parent_response = requests.get(parent_url).json()
    child_id = parent_response['childViews'][0]

    #Add the provided bin to the api query.
    url ='http://data.cityofnewyork.us/resource/{}.json?bin={}'.format(child_id,bin)
    # url = 'http://data.cityofnewyork.us/resource/7xhx-ywii.json?bin={}'.format(bin)

    #Encode the url, but allow the characters specified in the safe argument.
    url = quote(url, safe = ':/?&=')

    try:
        #Send the get request to Socrata and return the results as a json
        response = json.loads(json.dumps(requests.get(url).json()))
        
        if len(response) > 0:
            #Iterate through the items in the response json
            for r in response:
                #Convert the geometry value to a shapely geometry
                r[geom_col] = shape(r[geom_col])
                #Append the response to the address points list
                lpc.append(r)
        
        else:
            #lpc.append({'bin':bin, geom_col: np.nan})
            pass

    except:
        #Append only the bin for records where the bins don't have a corresponding address point
        #add_pts.append([{'bin':bin}])
        failed.append(bin)

    return lpc

In [ ]:
def get_lpc_district(bin, geom_col):
    failed = []
    lpc = []
    parent_url = 'https://data.cityofnewyork.us/api/views/ts56-fkf5'
    parent_response = requests.get(parent_url).json()
    child_id = parent_response['childViews'][0]

    #Add the provided bin to the api query.
    url ='http://data.cityofnewyork.us/resource/{}.json?bin={}'.format(child_id,bin)
    # url = 'http://data.cityofnewyork.us/resource/7xhx-ywii.json?bin={}'.format(bin)

    #Encode the url, but allow the characters specified in the safe argument.
    url = quote(url, safe = ':/?&=')

    try:
        #Send the get request to Socrata and return the results as a json
        response = json.loads(json.dumps(requests.get(url).json()))
        
        if len(response) > 0:
            #Iterate through the items in the response json
            for r in response:
                #Convert the geometry value to a shapely geometry
                r[geom_col] = shape(r[geom_col])
                #Append the response to the address points list
                lpc.append(r)
        
        else:
            #lpc.append({'bin':bin, geom_col: np.nan})
            pass

    except:
        #Append only the bin for records where the bins don't have a corresponding address point
        #add_pts.append([{'bin':bin}])
        failed.append(bin)

    return lpc

In [ ]:
def get_lpc_individual_historic(bin, geom_col):
    failed = []
    lpc = []
    parent_url = 'https://data.cityofnewyork.us/api/views/7mgd-s57w'
    parent_response = requests.get(parent_url).json()
    child_id = parent_response['childViews'][0]

    #Add the provided bin to the api query.
    url ='http://data.cityofnewyork.us/resource/{}.json?bin={}'.format(child_id,bin)
    # url = 'http://data.cityofnewyork.us/resource/7xhx-ywii.json?bin={}'.format(bin)

    #Encode the url, but allow the characters specified in the safe argument.
    url = quote(url, safe = ':/?&=')

    try:
        #Send the get request to Socrata and return the results as a json
        response = json.loads(json.dumps(requests.get(url).json()))
        
        if len(response) > 0:
            #Iterate through the items in the response json
            for r in response:
                #Convert the geometry value to a shapely geometry
                r[geom_col] = shape(r[geom_col])
                #Append the response to the address points list
                lpc.append(r)
        
        else:
            #lpc.append({'bin':bin, geom_col: np.nan})
            pass

    except:
        #Append only the bin for records where the bins don't have a corresponding address point
        #add_pts.append([{'bin':bin}])
        failed.append(bin)

    return lpc

In [20]:
url ='http://data.cityofnewyork.us/resource/{}.json'.format('ncre-qhxs')

#Encode the url, but allow the characters specified in the safe argument.
url = quote(url, safe = ':/?&=')


#Send the get request to Socrata and return the results as a json
response = json.loads(json.dumps(requests.get(url).json()))

In [27]:
keep_cols = ['lp_number', 'lm_name', 'lm_type', 'status', 'last_actio', 'desdate', 'caldate', 'the_geom']

In [28]:
lm_df = (pd.DataFrame(response)[keep_cols]
         .rename(columns = {'bin_number': 'bin',
                            'status': 'lm_status',
                            'last_actio': 'lm_last_action',
                            'desdate': 'lm_desdate',
                            'caldate': 'lm_caldate'}))

In [29]:
lm_df

,lp_number,lm_name,lm_type,lm_status,lm_last_action,lm_desdate,lm_caldate,the_geom
0,LP-00861,Carnegie Hill Historic District,Historic District,DESIGNATED,DESIGNATED,1974-07-23T00:00:00.000,NaN,"{'type': 'Point', 'coordinates': [996449.41457..."
1,LP-02443,Park Slope Historic District Extension,Historic District,DESIGNATED,DESIGNATED,2012-04-17T00:00:00.000,2010-08-10T00:00:00.000,"{'type': 'Point', 'coordinates': [989224.87651..."
2,LP-02448,Central Ridgewood Historic District,Historic District,DESIGNATED,DESIGNATED,2014-12-09T00:00:00.000,2010-10-26T00:00:00.000,"{'type': 'Point', 'coordinates': [1011709.5943..."
3,LP-02361,Crown Heights North Historic District II,Historic District,DESIGNATED,DESIGNATED,2011-06-28T00:00:00.000,2009-06-23T00:00:00.000,"{'type': 'Point', 'coordinates': [998461.81638..."
4,LP-01898,Clay Avenue Historic District,Historic District,DESIGNATED,DESIGNATED,1994-04-05T00:00:00.000,NaN,"{'type': 'Point', 'coordinates': [1008388.6403..."
...,...,...,...,...,...,...,...,...
995,LP-01088,Chelsea Historic District Extension,Historic District,DESIGNATED,DESIGNATED,1981-02-03T00:00:00.000,NaN,"{'type': 'Point', 'coordinates': [983460.27552..."
996,LP-01957,Douglaston Historic District,Historic District,DESIGNATED,DESIGNATED,1997-06-24T00:00:00.000,1996-11-19T00:00:00.000,"{'type': 'Point', 'coordinates': [1053696.5254..."
997,LP-01051,Upper East Side Historic District,Historic District,DESIGNATED,DESIGNATED,1981-05-19T00:00:00.000,NaN,"{'type': 'Point', 'coordinates': [994475.56811..."
998,LP-01712,Tribeca South Historic District,Historic District,DESIGNATED,DESIGNATED,1992-12-08T00:00:00.000,NaN,"{'type': 'Point', 'coordinates': [982285.01442..."


In [5]:
def get_lpc(bin, geom_col):
    failed = []
    lpc = []
    parent_id = 'ncre-qhxs'

    #Add the provided bin to the api query.
    url ='http://data.cityofnewyork.us/resource/{}.json?bin_number={}'.format(parent_id,bin)
    # url = 'http://data.cityofnewyork.us/resource/7xhx-ywii.json?bin={}'.format(bin)

    #Encode the url, but allow the characters specified in the safe argument.
    url = quote(url, safe = ':/?&=')

    try:
        #Send the get request to Socrata and return the results as a json
        response = json.loads(json.dumps(requests.get(url).json()))
        
        if len(response) > 0:
            #Iterate through the items in the response json
            for r in response:
                #Convert the geometry value to a shapely geometry
                r[geom_col] = shape(r[geom_col])
                #Append the response to the address points list
                lpc.append(r)
        
        else:
            #lpc.append({'bin':bin, geom_col: np.nan})
            pass

    except:
        #Append only the bin for records where the bins don't have a corresponding address point
        #add_pts.append([{'bin':bin}])
        failed.append(bin)

    return lpc

In [6]:
lpc = structures.apply(lambda x: get_lpc(x['bin'], 'the_geom'), axis = 1)

In [7]:
lpc_unlist = [inner for outer in lpc for inner in outer]

In [8]:
keep_cols = ['lp_number', 'lm_name', 'lm_type', 'status', 'last_actio', 'desdate', 'caldate', 'the_geom']

In [9]:
lm_df = (pd.DataFrame(lpc_unlist)[keep_cols]
         .rename(columns = {'bin_number': 'bin',
                            'status': 'lm_status',
                            'last_actio': 'lm_last_action',
                            'desdate': 'lm_desdate',
                            'caldate': 'lm_caldate'})
         .astype({'bin': 'int64'}))

In [ ]:
sjoin = gpd.sjoin(structures,lm_df)
output = sjoin[columns]

In [11]:
lm_df_final = lm_df.merge(structures, how = 'left', on = 'bin').drop(columns = 'bin')

In [16]:
lm_df_final[lm_df_final['system'] == 'M010-ZN01-BLG0707']

,lp_number,lm_name,lm_type,lm_status,lm_last_action,lm_desdate,lm_caldate,system
12,LP-00851,Central Park,Scenic Landmark,DESIGNATED,DESIGNATED,1974-04-16T00:00:00.000,NaN,M010-ZN01-BLG0707
26,LP-00851,Central Park,Scenic Landmark,DESIGNATED,DESIGNATED,1974-04-16T00:00:00.000,NaN,M010-ZN01-BLG0707
48,LP-00851,Central Park,Scenic Landmark,DESIGNATED,DESIGNATED,1974-04-16T00:00:00.000,NaN,M010-ZN01-BLG0707
123,LP-00851,Central Park,Scenic Landmark,DESIGNATED,DESIGNATED,1974-04-16T00:00:00.000,NaN,M010-ZN01-BLG0707
134,LP-00851,Central Park,Scenic Landmark,DESIGNATED,DESIGNATED,1974-04-16T00:00:00.000,NaN,M010-ZN01-BLG0707
155,LP-00851,Central Park,Scenic Landmark,DESIGNATED,DESIGNATED,1974-04-16T00:00:00.000,NaN,M010-ZN01-BLG0707
180,LP-00851,Central Park,Scenic Landmark,DESIGNATED,DESIGNATED,1974-04-16T00:00:00.000,NaN,M010-ZN01-BLG0707
283,LP-00851,Central Park,Scenic Landmark,DESIGNATED,DESIGNATED,1974-04-16T00:00:00.000,NaN,M010-ZN01-BLG0707
